# **Mantenimiento de Datos OCDE en el SGBD**

* El Siguiente programa extraerá los datos de la OCDE por medio de una API y generará una consulta en el SGBD de tres tablas alojados al SGBD: investigadores_mujeres_OCDE, investigadores_1000 y investigadores_SCIMago, los resultados de ambos procedimientos se compararan los datos y si existen diferencia se generará un registro de sus respectivas diferencias.

* En el caso de existir diferencias el programa modificará el dato al mas actualizado.

In [2]:
# Importamos las librerias necesarias
import requests
import openpyxl # para exportar los resultados a un excel
import csv
import os
import pandas as pd

## Obtenemos los indicadores de ciencia y tecnología OCDE cada mil personas trabajando desde el 2012
https://data-explorer.oecd.org/vis?tm=main%20science&pg=0&snb=1&vw=tb&df[ds]=dsDisseminateFinalDMZ&df[id]=DSD_MSTI%40DF_MSTI&df[ag]=OECD.STI.STP&df[vs]=1.1&pd=2012%2C&dq=.A.T_RS.10P3EMP..&ly[rw]=REF_AREA&ly[cl]=TIME_PERIOD&to[TIME_PERIOD]=false

In [26]:
nombre_del_archivo = "OCDE_KTrabajadores.csv"                             # Nombre del archivo a buscar
carpeta = os.getcwd()                                       # Utiliza el directorio actual de trabajo como carpeta de búsqueda
ruta_completa = os.path.join(carpeta, nombre_del_archivo)   # Construcción de la ruta del archivo

# Verificar si el archivo existe en la carpeta
if os.path.isfile(ruta_completa):
    print(f"El archivo '{nombre_del_archivo}' está presente en el directorio actual.")
else:
    print(f"El archivo '{nombre_del_archivo}' no se encontró en el directorio actual.")
    print("Procedemos a conectarnos con la API para obtener el descargable.")

    # Define the URL of the request
    url = "https://sdmx.oecd.org/public/rest/data/OECD.STI.STP,DSD_MSTI@DF_MSTI,1.1/.A.T_RS.10P3EMP..?startPeriod=2012&dimensionAtObservation=AllDimensions"

    # Establece las cabeceras para solicitar datos formato csv
    parametros = {
        'Accept': 'application/vnd.sdmx.data+csv; charset=utf-8',
    }

    # Realizar la solicitud GET
    response = requests.get(url, headers=parametros)

    # Verifique el código de estado de la respuesta
    if response.status_code == 200:
        # Si la respuesta está en formato CSV, puede guardarla en un archivo o procesarla según sea necesario
        with open(nombre_del_archivo, 'w') as file:
            file.write(response.text)
        print("Descarga Exitosa")
    else:
        print(f"Error de solicitud: código de estado {response.status_code}")

def extraccion_ocde(ruta_archivo_csv):
    with open(ruta_archivo_csv, newline='', encoding='utf-8') as archivo_csv:
        lector = csv.reader(archivo_csv)
        datos = list(lector)

    datos_por_pais = {}
    for fila in datos[1:]:  # Ignora el encabezado
        if len(fila) > 8:  # Asegura que la fila tenga suficientes columnas
            pais = fila[1]  # La abreviatura del país está en la segunda columna
            ano = fila[7]  # El año está en la octava columna
            valor = fila[8]  # El dato está en la novena columna
            if pais not in datos_por_pais:
                datos_por_pais[pais] = {}
            try:
                datos_por_pais[pais][ano] = float(valor)
            except ValueError:
                continue  # Ignora los valores que no se pueden convertir a flotante

    return datos_por_pais

# Lista de códigos ISO 3166-1 alfa-3 de países y sus nombres completos para mostrar
paises_ocde = [
    ("AUT", "Austria"), ("BEL", "Bélgica"), ("CAN", "Canadá"), ("CHL", "Chile"), ("CRI", "Costa Rica"), 
    ("CZE", "República Checa"), ("DNK", "Dinamarca"), ("EST", "Estonia"), ("FIN", "Finlandia"),
    ("FRA", "Francia"), ("DEU", "Alemania"), ("GRC", "Grecia"), ("HUN", "Hungría"), ("ISL", "Islandia"),
    ("IRL", "Irlanda"), ("ITA", "Italia"), ("JPN", "Japón"), ("KOR", "Corea del Sur"),
    ("LVA", "Letonia"), ("LTU", "Lituania"), ("LUX", "Luxemburgo"), ("MEX", "México"), ("NLD", "Países Bajos"),
    ("NZL", "Nueva Zelanda"), ("NOR", "Noruega"), ("POL", "Polonia"), ("PRT", "Portugal"), ("SVK", "Eslovaquia"),
    ("SVN", "Eslovenia"), ("ESP", "España"), ("SWE", "Suecia"), ("CHE", "Suiza"), ("TUR", "Turquía"),
    ("GBR", "Reino Unido"), ("USA", "Estados Unidos"), ("EU27_2020", "Unión Europea (27)") ,("OECD", "OECD - Total")
]

# Llamada a la función de extracción con la ruta del archivo CSV
datos_por_pais = extraccion_ocde(nombre_del_archivo)

# Convertir los datos extraídos a una lista de diccionarios para crear un DataFrame
lista_datos = []
for pais, datos_anuales in datos_por_pais.items():
    for ano, valor in datos_anuales.items():
        lista_datos.append({'Año': int(ano), 'País': pais, 'Valor': valor})

# Crear el DataFrame a partir de la lista de diccionarios
df_ocde_KTrabajadores = pd.DataFrame(lista_datos)

# Convertir la lista de tuplas de países en un diccionario
codigo_a_nombre = dict(paises_ocde)

# Mapear los códigos de país a nombres completos en el DataFrame
df_ocde_KTrabajadores['País'] = df_ocde_KTrabajadores['País'].map(codigo_a_nombre)

# Eliminar los registros donde 'País' es NaN
df_ocde_KTrabajadores.dropna(subset=['País'], inplace=True)

# Eliminar directamente los registros del año 2022
df_ocde_KTrabajadores = df_ocde_KTrabajadores[df_ocde_KTrabajadores['Año'] != 2022]

# Mostrar el DataFrame resultante
print(df_ocde_KTrabajadores)



El archivo 'OCDE_KTrabajadores.csv' está presente en el directorio actual.
      Año        País     Valor
0    2012      Canadá  9.131744
1    2013      Canadá  9.112551
2    2014      Canadá  9.003252
3    2015      Canadá  8.990800
4    2016      Canadá  8.688412
..    ...         ...       ...
392  2017  Eslovaquia  6.418203
393  2018  Eslovaquia  6.751194
394  2019  Eslovaquia  6.942987
395  2020  Eslovaquia  7.201290
396  2021  Eslovaquia  7.345063

[339 rows x 3 columns]


## Obtenemos los indicadores de ciencia y tecnología OCDE con respecto al PIB desde el 2017
https://data-explorer.oecd.org/vis?pg=0&bp=true&snb=6&tm=science&vw=tb&df[ds]=dsDisseminateFinalDMZ&df[id]=DSD_MSTI%40DF_MSTI&df[ag]=OECD.STI.STP&df[vs]=1.0&pd=2017%2C&dq=.A.G.PT_B1GQ..&ly[rw]=REF_AREA&ly[cl]=TIME_PERIOD&to[TIME_PERIOD]=false

In [27]:
nombre_del_archivo = "OCDE_PIB.csv"                             # Nombre del archivo a buscar
carpeta = os.getcwd()                                       # Utiliza el directorio actual de trabajo como carpeta de búsqueda
ruta_completa = os.path.join(carpeta, nombre_del_archivo)   # Construcción de la ruta del archivo

# Verificar si el archivo existe en la carpeta
if os.path.isfile(ruta_completa):
    print(f"El archivo '{nombre_del_archivo}' está presente en el directorio actual.")
else:
    print(f"El archivo '{nombre_del_archivo}' no se encontró en el directorio actual.")
    print("Procedemos a conectarnos con la API para obtener el descargable.")

    # Define the URL of the request
    url = "https://sdmx.oecd.org/public/rest/data/OECD.STI.STP,DSD_MSTI@DF_MSTI,1.0/.A.G.PT_B1GQ..?startPeriod=2017&dimensionAtObservation=AllDimensions"

    # Establece las cabeceras para solicitar datos formato csv
    parametros = {
        'Accept': 'application/vnd.sdmx.data+csv; charset=utf-8',
    }

    # Realizar la solicitud GET
    response = requests.get(url, headers=parametros)

    # Verifique el código de estado de la respuesta
    if response.status_code == 200:
        # Si la respuesta está en formato CSV, puede guardarla en un archivo o procesarla según sea necesario
        with open(nombre_del_archivo, 'w') as file:
            file.write(response.text)
        print("Descarga Exitosa")
    else:
        print(f"Error de solicitud: código de estado {response.status_code}")

def extraccion_ocde(ruta_archivo_csv):
    with open(ruta_archivo_csv, newline='', encoding='utf-8') as archivo_csv:
        lector = csv.reader(archivo_csv)
        datos = list(lector)

    datos_por_pais = {}
    for fila in datos[1:]:  # Ignora el encabezado
        if len(fila) > 8:  # Asegura que la fila tenga suficientes columnas
            pais = fila[1]  # La abreviatura del país está en la segunda columna
            ano = fila[7]  # El año está en la octava columna
            valor = fila[8]  # El dato está en la novena columna
            if pais not in datos_por_pais:
                datos_por_pais[pais] = {}
            try:
                datos_por_pais[pais][ano] = float(valor)
            except ValueError:
                continue  # Ignora los valores que no se pueden convertir a flotante

    return datos_por_pais

# Lista de códigos ISO 3166-1 alfa-3 de países y sus nombres completos para mostrar
paises_ocde = [
    ("AUT", "Austria"), ("BEL", "Bélgica"), ("CAN", "Canadá"), ("CHL", "Chile"), ("CRI", "Costa Rica"), 
    ("CZE", "República Checa"), ("DNK", "Dinamarca"), ("EST", "Estonia"), ("FIN", "Finlandia"),
    ("FRA", "Francia"), ("DEU", "Alemania"), ("GRC", "Grecia"), ("HUN", "Hungría"), ("ISL", "Islandia"),
    ("IRL", "Irlanda"), ("ITA", "Italia"), ("JPN", "Japón"), ("KOR", "Corea del Sur"),
    ("LVA", "Letonia"), ("LTU", "Lituania"), ("LUX", "Luxemburgo"), ("MEX", "México"), ("NLD", "Países Bajos"),
    ("NZL", "Nueva Zelanda"), ("NOR", "Noruega"), ("POL", "Polonia"), ("PRT", "Portugal"), ("SVK", "Eslovaquia"),
    ("SVN", "Eslovenia"), ("ESP", "España"), ("SWE", "Suecia"), ("CHE", "Suiza"), ("TUR", "Turquía"),
    ("GBR", "Reino Unido"), ("USA", "Estados Unidos"), ("EU27_2020", "Unión Europea (27)") ,("OECD", "OECD - Total")
]

# Llamada a la función de extracción con la ruta del archivo CSV
datos_por_pais = extraccion_ocde(nombre_del_archivo)

# Convertir los datos extraídos a una lista de diccionarios para crear un DataFrame
lista_datos = []
for pais, datos_anuales in datos_por_pais.items():
    for ano, valor in datos_anuales.items():
        lista_datos.append({'Año': int(ano), 'País': pais, 'Valor': valor})

# Crear el DataFrame a partir de la lista de diccionarios
df_ocde_pib = pd.DataFrame(lista_datos)

# Convertir la lista de tuplas de países en un diccionario
codigo_a_nombre = dict(paises_ocde)

# Mapear los códigos de país a nombres completos en el DataFrame
df_ocde_pib['País'] = df_ocde_pib['País'].map(codigo_a_nombre)

# Eliminar los registros donde 'País' es NaN
df_ocde_pib.dropna(subset=['País'], inplace=True)

# Eliminar directamente los registros del año 2022
df_ocde_pib = df_ocde_pib[df_ocde_pib['Año'] != 2022]

# Mostrar el DataFrame resultante
print(df_ocde_pib)

El archivo 'OCDE_PIB.csv' está presente en el directorio actual.
      Año             País     Valor
4    2017         Islandia  2.084135
5    2018         Islandia  2.000809
6    2019         Islandia  2.341648
7    2020         Islandia  2.490692
8    2021         Islandia  2.805417
..    ...              ...       ...
215  2021  República Checa  1.996006
216  2017       Costa Rica  0.430539
217  2018       Costa Rica  0.371300
218  2020       Costa Rica  0.330022
219  2021       Costa Rica  0.275308

[175 rows x 3 columns]


## Obtenemos los indicadores de ciencia y tecnología OCDE de investigadores e investigadoras desde el 2010
https://data-explorer.oecd.org/vis?tm=main%20science&pg=0&snb=1&vw=tb&df[ds]=dsDisseminateFinalDMZ&df[id]=DSD_MSTI%40DF_MSTI&df[ag]=OECD.STI.STP&df[vs]=1.1&pd=2010%2C&dq=.A.T_RS%2BT_WRS.PS..&ly[rw]=REF_AREA&ly[cl]=TIME_PERIOD&to[TIME_PERIOD]=false

In [44]:
nombre_del_archivo = "OCDE_Inv.csv"                             # Nombre del archivo a buscar
carpeta = os.getcwd()                                       # Utiliza el directorio actual de trabajo como carpeta de búsqueda
ruta_completa = os.path.join(carpeta, nombre_del_archivo)   # Construcción de la ruta del archivo

# Verificar si el archivo existe en la carpeta
if os.path.isfile(ruta_completa):
    print(f"El archivo '{nombre_del_archivo}' está presente en el directorio actual.")
else:
    print(f"El archivo '{nombre_del_archivo}' no se encontró en el directorio actual.")
    print("Procedemos a conectarnos con la API para obtener el descargable.")

    # Define the URL of the request
    url = "https://sdmx.oecd.org/public/rest/data/OECD.STI.STP,DSD_MSTI@DF_MSTI,1.1/.A.T_RS+T_WRS.PS..?startPeriod=2010&dimensionAtObservation=AllDimensions"

    # Establece las cabeceras para solicitar datos formato csv
    parametros = {
        'Accept': 'application/vnd.sdmx.data+csv; charset=utf-8',
    }

    # Realizar la solicitud GET
    response = requests.get(url, headers=parametros)

    # Verifique el código de estado de la respuesta
    if response.status_code == 200:
        # Si la respuesta está en formato CSV, puede guardarla en un archivo o procesarla según sea necesario
        with open(nombre_del_archivo, 'w') as file:
            file.write(response.text)
        print("Descarga Exitosa")
    else:
        print(f"Error de solicitud: código de estado {response.status_code}")

def extraccion_ocde(ruta_archivo_csv):
    with open(ruta_archivo_csv, newline='', encoding='utf-8') as archivo_csv:
        lector = csv.reader(archivo_csv)
        datos = list(lector)

    datos_por_pais_y_categoria = {}
    for fila in datos[1:]:  # Ignora el encabezado
        if len(fila) > 8:  # Asegura que la fila tenga suficientes columnas
            pais = fila[1]  # La abreviatura del país está en la segunda columna
            categoria = fila[3]  # La categoría (T_WRS o T_RS) está en la cuarta columna
            ano = fila[7]  # El año está en la octava columna
            valor = fila[8]  # El dato está en la novena columna
            if pais not in datos_por_pais_y_categoria:
                datos_por_pais_y_categoria[pais] = {}
            if categoria not in datos_por_pais_y_categoria[pais]:
                datos_por_pais_y_categoria[pais][categoria] = {}
            try:
                datos_por_pais_y_categoria[pais][categoria][ano] = float(valor)
            except ValueError:
                continue  # Ignora los valores que no se pueden convertir a flotante

    return datos_por_pais_y_categoria

# Lista de códigos ISO 3166-1 alfa-3 de países y sus nombres completos para mostrar
paises_ocde = [
    ("AUT", "Austria"), ("BEL", "Bélgica"), ("CAN", "Canadá"), ("CHL", "Chile"), ("CRI", "Costa Rica"), 
    ("CZE", "República Checa"), ("DNK", "Dinamarca"), ("EST", "Estonia"), ("FIN", "Finlandia"),
    ("FRA", "Francia"), ("DEU", "Alemania"), ("GRC", "Grecia"), ("HUN", "Hungría"), ("ISL", "Islandia"),
    ("IRL", "Irlanda"), ("ITA", "Italia"), ("JPN", "Japón"), ("KOR", "Corea del Sur"),
    ("LVA", "Letonia"), ("LTU", "Lituania"), ("LUX", "Luxemburgo"), ("MEX", "México"), ("NLD", "Países Bajos"),
    ("NZL", "Nueva Zelanda"), ("NOR", "Noruega"), ("POL", "Polonia"), ("PRT", "Portugal"), ("SVK", "Eslovaquia"),
    ("SVN", "Eslovenia"), ("ESP", "España"), ("SWE", "Suecia"), ("CHE", "Suiza"), ("TUR", "Turquía"),
    ("GBR", "Reino Unido"), ("USA", "Estados Unidos"), ("EU27_2020", "Unión Europea (27)") ,("OECD", "OECD - Total")
]

# Llamada a la función de extracción con la ruta del archivo CSV
datos_por_pais_y_categoria = extraccion_ocde(nombre_del_archivo)

# Mostrar resultados para pais, categoría y calcula la proporción de investigadoras
lista_datos = []

for codigo_pais, nombre_pais in paises_ocde:
    if codigo_pais in datos_por_pais_y_categoria:
        datos_pais = datos_por_pais_y_categoria[codigo_pais]
        for ano in range(2010, 2022):  # Asume que los datos están disponibles para estos años
            valor_investigadores = datos_pais.get('T_RS', {}).get(str(ano))
            valor_investigadoras = datos_pais.get('T_WRS', {}).get(str(ano))
            if valor_investigadores and valor_investigadoras:
                proporcion = (valor_investigadoras / valor_investigadores) * 100
                lista_datos.append({'Año': ano, 'País': nombre_pais, 'Proporción': proporcion})

# Crear el DataFrame a partir de la lista
df_ocde_inv = pd.DataFrame(lista_datos)

# Eliminar los registros donde 'País' es NaN y del año 2022 en adelante
df_ocde_inv.dropna(subset=['País'], inplace=True)
df_ocde_inv = df_ocde_inv[df_ocde_inv['Año'] < 2022]

print(df_ocde_inv)        


El archivo 'OCDE_Inv.csv' está presente en el directorio actual.
      Año         País  Proporción
0    2011      Austria   28.989925
1    2013      Austria   29.594950
2    2015      Austria   29.493536
3    2017      Austria   30.059296
4    2019      Austria   30.392041
..    ...          ...         ...
302  2013  Reino Unido   38.098367
303  2014  Reino Unido   37.411822
304  2015  Reino Unido   38.589967
305  2016  Reino Unido   38.666093
306  2017  Reino Unido   38.749098

[307 rows x 3 columns]


In [45]:
'''
# Especifica el nombre del archivo Excel donde deseas guardar los resultados
nombre_archivo_excel = 'Datos_OCDE.xlsx'

# Guarda el DataFrame en un archivo Excel
df_ocde_inv.to_excel(nombre_archivo_excel, index=False, engine='openpyxl')

print(f'Los datos han sido guardados con éxito en {nombre_archivo_excel}')
'''

Los datos han sido guardados con éxito en Datos_OCDE.xlsx
